# MNIST in PyTorch

- Based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In [0]:
import os

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

***

## Define model etc.

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [0]:
def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [0]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

***

## Settings

In [0]:
batch_size=64
test_batch_size=1000
epochs = 14
lr = 1
gamma = 0.7
seed = 1
log_interval = 200

data_dir = os.path.join(os.getcwd(), 'data')

In [7]:
use_cuda = torch.cuda.is_available()
print('Using GPU...' if use_cuda else 'NOT using GPU...')

device = torch.device('cuda:0' if use_cuda else 'cpu')
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

Using GPU...


In [8]:
torch.manual_seed(seed)

***

## Get dataset

In [9]:
# Since torchvision v0.2.1
datasets.MNIST(data_dir, download=True)

0it [00:00, ?it/s]

9920512it [00:01, 8616345.72it/s]                            


Extracting /content/data/MNIST/raw/train-images-idx3-ubyte.gz to /content/data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 129094.90it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

...

1654784it [00:00, 2130695.19it/s]                            
0it [00:00, ?it/s]

Extracting /content/data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/data/MNIST/raw


8192it [00:00, 49305.67it/s]            


Extracting /content/data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/data/MNIST/raw
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: /content/data
    Split: Train

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=batch_size, shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        data_dir, train=False, 
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
    batch_size=test_batch_size, shuffle=True, num_workers=2)

***

## Start training

In [0]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [12]:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(0, epochs):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.327989
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.326843
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.185185
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.145754
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.216581

Test set: Average loss: 0.0583, Accuracy: 9811/10000 (98%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.158418
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.016682
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.014943
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.080406
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.201064

Test set: Average loss: 0.0396, Accuracy: 9875/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.048407
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.046408
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.014332
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.044078
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.018092

Test set: Average loss: 0.0344, Accuracy: 9895/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.028440
Train Epoch: 3 [12800/60000 (21%)]	Lo

***

## Save trained model

In [0]:
torch.save(model.state_dict(), "mnist_cnn.pt")